# Inside Quantum Classifiers

This tutorial offers a deep dive in the internals of quantum classifiers and several exercises on solving a simple classification problem from scratch. You will learn how to build a simple classifier by hand and how to train a simple model using the [quantum machine learning library](https://docs.microsoft.com/quantum/libraries/machine-learning/) that is part of the Microsoft Quantum Development Kit. 

The companion Python notebook [Exploring Quantum Classification Library](./ExploringQuantumClassificationLibrary.ipynb) offers a high-level walkthrough of solving the same classification problem.

### Prerequisites

We recommend you to get familiar with the basics of quantum computing and quantum programming before attempting this tutorial. It relies on the concepts of superposition and measurement, as well as the basic knowledge of quantum gates and circuits.

* You can use the [tutorials](https://github.com/Microsoft/QuantumKatas/#list-of-tutorials-) offered by the Quantum Katas project to learn the basics and their representation in Q#.
* Alternatively, [Microsoft Quantum Development Kit documentation](https://docs.microsoft.com/quantum) covers a lot of introductory topics.

To start, execute the cell below to load the [Microsoft.Quantum.MachineLearning package](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.machinelearning) 
which provides the necessary functionality.

In [23]:
%package Microsoft.Quantum.MachineLearning::0.11.2004.2825

Adding package Microsoft.Quantum.MachineLearning::0.11.2004.2825: done!

Microsoft.Quantum.Standard::0.11.2004.2825, Microsoft.Quantum.MachineLearning::0.11.2004.2825

## Circuit-centric Quantum Classifiers: an Overview

Circuit-centric quantum classifiers have the following structure:

1. Data encoding.  
The features of a data sample are encoded as the amplitudes of a quantum state. This state further serves as the input to the classification circuit. The encoding is parameter-free and uses only the raw data of the sample.

2. Classification model circuit.  
The classification circuit is a small-depth circuit of single-qubit rotations and two-qubit controlled rotations. 
The circuit geometry (the sequence of the rotations and the qubits to which they are applied) is fixed for the model, and the parameters of the model (the rotation angles) are learned during the training phase.

3. Measurement.  
Applying the classification circuit to the data encoded as a quantum state yields the final quantum state. 
Measuring the "output" qubit of that state allows us to get a classical result - 0 or 1. 
However, that is not the final classification result yet...

4. Result interpretation.  
The steps 1-3 are repeated multiple times to estimate the probability of getting 0 or 1 in the final measurement.
This probability is compared with a threshold (classifier bias) to produce the final classification result.

You can read more about the theory behind curcuit-centric quantum classifiers in the [Microsoft QDK documentation](https://docs.microsoft.com/quantum/libraries/machine-learning/introduction) or in the [original paper](https://arxiv.org/abs/1804.00633). 
In this tutorial we will focus on walking through each step of the classification and training process and implementing them.

## The Raw Data

Same as in the companion tutorial, we start by preparing the training and validation datasets. 

> Q# is a domain-specific programming language, designed to express quantum algorithms, but it supports a subset of classical language features sufficient for performing simple computations, such as data generation. When solving a real problem, you'll want to load the data from an external source before calling Q# code to process it.

This Q# code follows the same data generation logic as the [Python code](./ExploringQuantumClassificationLibrary.ipynb#The-Data) in the companion notebook. 
There are two features, each of them is a real number from $[0, 1)$ range.

In [18]:
open Microsoft.Quantum.Math;

operation GenerateData (samplesNumber : Int, separationAngles : Double[]) : (Double[][], Int[]) {
    mutable features = new Double[][samplesNumber];
    mutable labels = new Int[samplesNumber];
    for (i in 0 .. samplesNumber - 1) {
        let sample = [RandomReal(8), RandomReal(8)];
        let angle = ArcTan2(sample[1], sample[0]);
        set features w/= i <- sample;
        set labels w/= i <- (angle < separationAngles[0] or angle > separationAngles[1]) ? 0 | 1;
    }
    return (features, labels);
}

operation SampleDataDemo () : Unit {
    let trainingData = GenerateData(5, [PI() / 6.0, PI()/ 3.0]);
    Message($"{trainingData}");
}

GenerateData, SampleDataDemo

`%simulate` cells run the Q# code on a quantum simulator. Executing the following cell shows a small sample of data generated by the code above.

In [19]:
%simulate SampleDataDemo

([[0.84765625,0.48046875],[0.6875,0.40234375],[0.03515625,0.3828125],[0.91796875,0.73046875],[0.8515625,0.453125]], [0,1,0,1,0])


()

## Data Encoding

The first step of the quantum classification process is encoding the raw feature data into the amplitudes of a quantum state. 

> If you need a refresher on quantum state representation, see [Multi-qubit Systems tutorial](https://github.com/microsoft/QuantumKatas/tree/master/tutorials/MultiQubitSystems).

An $n$-qubit quantum state can be described by $2^n$ amplitudes. 
If the data has $M$ features, it can be encoded in the amplitudes of a state with $n = \lceil \log_2 M \rceil$ qubits. 
In general case the amplitudes can be complex, but for the purposes of data encoding it's sufficient to use only real amplitudes.

In our case $M = 2$, so we can encode the features $(x_0, x_1)$ in the state of one qubit as the amplitudes of the basis states $|0\rangle$ and $|1\rangle$, respectively. 
The sum of squares of the amplitudes of the basis states has to be 1, so we'll have to normalize our data:

$$(x_0, x_1) \rightarrow |\psi(x_0, x_1) \rangle = \tilde{x}_0 |0\rangle + \tilde{x}_1 |1\rangle \text{, where }
\tilde{x}_0 = \frac{x_0}{\sqrt{x_0^2 + x_1^2}}\text{ and }\tilde{x}_1 = \frac{x_1}{\sqrt{x_0^2 + x_1^2}}$$

> Note that this encoding will lose part of the information: if we plot the data, multiple points $(x_0, x_1)$ will be encoded in the same $|\psi(x_0, x_1) \rangle$. Effectively only the angular data is preserved, and the data about the distance to the origin is lost. 
> <img src="./img/1-data-encoding.PNG" width=300 alt="Two distinct data points are encoded as the same state" />
> If we need to preserve information about the distance to the origin (which is typically the case), we need to pre-process the data, adding an extra feature.
> In this tutorial we'll omit this step for simplicity; the synthetic data we're using is chosen so that only the angular data defines the classification of the sample.

To implement data encoding in Q#, we can use library routines 
[InputEncoder](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.machinelearning.inputencoder) or 
[ApproximateInputEncoder](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.machinelearning.approximateinputencoder).
Here is what the results will look like for one sample:

In [44]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.MachineLearning;

operation EncodeDataDemo () : Unit {
    let sample = [RandomReal(8), RandomReal(8)];
    Message($"Raw data: {sample}");
    
    let norm = Sqrt(sample[0] ^ 2.0 + sample[1] ^ 2.0);
    Message($"Normalized data: [{sample[0] / norm}, {sample[1] / norm}]");
    
    using (q = Qubit()) {
        let (_, encoder) = (InputEncoder(sample))!;
        encoder(LittleEndian([q]));
        Message("Encoded as a quantum state:");
        DumpMachine();
        Reset(q);
    }
}

EncodeDataDemo

In [46]:
%simulate EncodeDataDemo

Raw data: [0.0703125,0.56640625]
Normalized data: [0.12319234904975414, 0.9923828117896861]
Encoded as a quantum state:


|0⟩	0.12319234904975411 + 0𝑖
|1⟩	0.9923828117896862 + 0𝑖

()